In [1]:
import os, sys
from tqdm import tqdm
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import time

from src.features import intended_receiver, extract_player_pos, extract_pass

# this is very useful as it makes sure that always all columns and rows of a data frame are printed
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# datatools Animator
from datatools.metrica_helper import MetricaHelper
from datatools.trace_animator import TraceAnimator
from datatools.trace_helper import TraceHelper
import datatools.matplotsoccer as mps
plt.rcParams['animation.ffmpeg_path'] ='"C:\\Users\\M207\\Desktop\\kang\\code_factory\\EPV_intended\\ffmpeg\\bin\\ffmpeg.exe"'
from matplotlib import animation

### Data Loading

In [2]:
# data loading
match1 = pd.read_csv('../metrica-data/EPV-data/match1.csv')
match2 = pd.read_csv('../metrica-data/EPV-data/match2.csv')
match3 = pd.read_csv('../metrica-data/EPV-data/match3.csv')
all_events_unsuccess = pd.read_csv('../metrica-data/EPV-data/all-match-unsuccess.csv')

### Extracting video (Unsuccess pass)

In [3]:
# 실패한 패스 라벨링을 위해 영상 추출 func.
def unsuccess_to_video(helper, match_number, start_frame, end_frame):
    traces = helper.traces[start_frame-3:end_frame+5]
    animator = TraceAnimator(
        trace_dict={"main": traces},
        show_episodes=True,
        show_events=True
    )
    anim = animator.run()
    ani_html = anim.to_jshtml()

    frame_event = helper.events
    pass_id = frame_event[frame_event['start_frame'] == start_frame]['pass_id'].iloc[0]

    with open(f'../animation/match{match_number}_{pass_id}.html', 'w') as f:
        f.write(ani_html)

In [ ]:
# Extracting for loop
match_list = [match1, match2, match3]

for i in [1, 2, 3]:
    events = all_events_unsuccess[all_events_unsuccess['game_id']==i].copy()
    start_end = events[['start_frame', 'end_frame']]
    helper = MetricaHelper(traces_preprocessed=match_list[i-1], events=events)
    count = 0
    for index, row in tqdm(start_end.iterrows()):
        unsuccess_to_video(helper, i, row['start_frame'], row['end_frame'])
        count += 1
    print(f'match {i} 의 wrong pass : {count}')